In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import attila_utils

In [2]:
def read_results(fpath='../../results/experimental/cell-based-assays.xlsx', sheet='Sheet2'):
    results = pd.read_excel(fpath, sheet, header=[0, 1, 2])
    results = results.set_index(('class', 'assay', 'concentration'))
    results = results.drop(results.columns[0], axis=1)
    results = results.transpose()
    results = results.rename_axis(['class', 'assay', 'concentration'], axis='index')
    results['conc'] = results.index.get_level_values('concentration')
    results = results.rename(index={'c4': 'c1', 'c5': 'c2', 'c6': 'c3'}, level=2)
    return(results)

fpath = '../../resources/cell-based-assays/CO25360_Overview results_all-clean1.xlsx'
read_results(fpath, sheet='Sheet1')

('class', 'assay', 'concentration')                                                VALPROIC ACID  \
class                                          assay                concentration                  
Exp2 Abeta induced toxicity primary neurons    MTT (viability)      c1                       0.0   
                                                                    c2                       0.0   
                                                                    c3                       0.0   
Exp3 Abeta Clearance BV2                       Abeta in SN          c1                       0.0   
                                                                    c2                       1.0   
...                                                                                          ...   
Exp8 trophic factor withdrawal primary neurons LDH (cell death)     c2                       0.0   
                                                                    c3                       0.0   
Exp9 Tau Agg cell-free                         Tau aggregation ΔRFU c1                       0.0   
                                                                    c2                       0.0   
                                                                    c3                      -2.0   

('class', 'assay', 'concentration')                                                SALBUTAMOL  \
class                                          assay                concentration               
Exp2 Abeta induced toxicity primary neurons    MTT (viability)      c1                    0.0   
                                                                    c2                    0.0   
                                                                    c3                    0.0   
Exp3 Abeta Clearance BV2                       Abeta in SN          c1                    0.0   
                                                                    c2                    0.0   
...                                                                                       ...   
Exp8 trophic factor withdrawal primary neurons LDH (cell death)     c2                    0.0   
                                                                    c3                    0.0   
Exp9 Tau Agg cell-free                         Tau aggregation ΔRFU c1                    0.0   
                                                                    c2                    0.0   
                                                                    c3                    0.0   

('class', 'assay', 'concentration')                                                SILDENAFIL  \
class                                          assay                concentration               
Exp2 Abeta induced toxicity primary neurons    MTT (viability)      c1                    0.0   
                                                                    c2                    0.0   
                                                                    c3                    0.0   
Exp3 Abeta Clearance BV2                       Abeta in SN          c1                    0.0   
                                                                    c2                    0.0   
...                                                                                       ...   
Exp8 trophic factor withdrawal primary neurons LDH (cell death)     c2                    0.0   
                                                                    c3                    0.0   
Exp9 Tau Agg cell-free                         Tau aggregation ΔRFU c1                    0.0   
                                                                    c2                    0.0   
                                                                    c3                   -1.0   

('class', 'assay', 'concentration')                                                PROBENECID  \
class                                          assay                concentrat

In [3]:
fpath = '../../resources/cell-based-assays/CO28151_data-summary-clean.xlsx'
data = dict()
key = 'Abeta Clearance'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,Cohort,Group,IRN,TI,conc,Aβ42 SN,Aβ42 Lys,BCA,Aβ42 Lys pg per ug prot
0,plate1,A1,TI2 c1_1,TI2,C1,35,693.0,271.0,2.55
1,plate1,A1,TI2 c1_2,TI2,C1,102,948.0,454.0,2.09
2,plate1,A1,TI2 c1_3,TI2,C1,51,1885.0,474.0,3.98
3,plate1,A1,TI2 c1_4,TI2,C1,37,674.0,298.0,2.26
4,plate1,A1,TI2 c1_5,TI2,C1,55,430.0,336.0,1.28
...,...,...,...,...,...,...,...,...,...
300,Plate 1,VC1,VC_1_6,VC_1,NaN,60759,3023.0,371.0,8.15
301,Plate 2,VC2,VC_2_1,VC_2,NaN,64832,1492.0,409.0,3.65
302,Plate 2,VC2,VC_2_2,VC_2,NaN,53839,456.0,451.0,1.01
303,Plate 2,VC2,VC_2_3,VC_2,NaN,58911,195.0,410.0,0.48


In [4]:
key = 'Abeta secretion'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,plate,Group,IRN,TI,conc,Aβ38,Aβ40,Aβ42,ratio 42:40x100
0,1,A1,TI2 c1_1,2,1.0,4010.000,834290.00,67940.0000,8.100000
1,1,A1,TI2 c1_2,2,1.0,6280.000,784080.00,70540.0000,9.000000
2,1,A1,TI2 c1_3,2,1.0,27790.000,932160.00,81130.0000,8.700000
3,1,A1,TI2 c1_4,2,1.0,32560.000,878050.00,69370.0000,7.900000
4,1,A1,TI2 c1_5,2,1.0,20670.000,624520.00,48540.0000,7.800000
...,...,...,...,...,...,...,...,...,...
277,5,NaN,TI15 c3_2,15,3.0,2538.531,14177.22,979.6982,6.910369
278,5,NaN,TI15 c3_3,15,3.0,2677.243,14218.80,1072.2790,7.541276
279,5,NaN,TI15 c3_4,15,3.0,2782.730,13733.23,1086.8330,7.913892
280,5,NaN,TI15 c3_5,15,3.0,2530.309,13473.46,1070.6570,7.946415


In [5]:
key = 'LPS inflammation'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,Cohort,Group,IRN,TI,conc,IL-10,IL-1β,IL-6,KC/GRO,TNF-α
0,plate1,A1,TI2 c1_1,TI2,C1,1.46,1.25,1959,2.3,166
1,plate1,A1,TI2 c1_2,TI2,C1,3.23,2.08,3867,4.88,338
2,plate1,A1,TI2 c1_3,TI2,C1,2.81,2.29,4313,6.14,322
3,plate1,A1,TI2 c1_4,TI2,C1,2.57,1.75,4092,6.81,285
4,plate1,A1,TI2 c1_5,TI2,C1,3.15,2.26,2259,3.78,230
...,...,...,...,...,...,...,...,...,...,...
434,Plate 4,VC4,VC 4_2,VC 4,4,2.2,0.015,n.m.,n.m.,15
435,Plate 4,VC4,VC 4_3,VC 4,4,2.34,n.m.,n.m.,n.m.,18
436,Plate 4,VC4,VC 4_4,VC 4,4,1.94,n.m.,n.m.,n.m.,13
437,Plate 4,VC4,VC 4_5,VC 4,4,2.05,n.m.,n.m.,n.m.,19


In [6]:
key = 'Tau phosphorylation'
data[key] = pd.read_excel(fpath, sheet_name=key)
data[key]

,Batch,Plate,IRN,TI,Conc,Tau [pg/ml],pTau (T231) [AU],BCA total protein,Tau [pg/µg],pTau (T231) [AU2],Ratio pTau /Tau
0,batch 1,Plate 5,TI1 c1_1,TI1,1,508,62.4,240.3,2.1,29.5,14
1,batch 1,Plate 5,TI1 c1_2,TI1,1,322.8,47.6,185.6,1.7,27.4,15.7
2,batch 1,Plate 5,TI1 c1_3,TI1,1,338,45.9,158.3,2.1,21.5,10.1
3,batch 1,Plate 5,TI1 c1_4,TI1,1,339.6,49,161.8,2.1,23.4,11.1
4,batch 1,Plate 5,TI1 c1_5,TI1,1,334.2,47,320.3,1,45,43.1
...,...,...,...,...,...,...,...,...,...,...,...
361,batch 1,Plate 5,VC_5_2,VC,5,735.3,101.9,218.3,3.4,30.2,9
362,batch 1,Plate 5,VC_5_3,VC,5,912.1,125,244.3,3.7,33.5,9
363,batch 1,Plate 5,VC_5_4,VC,5,747,110.2,222.6,3.4,32.8,9.8
364,batch 1,Plate 5,VC_5_5,VC,5,2078.7,222.9,196.1,10.6,21,2


In [7]:
groups = {k: v.TI.unique() for k, v in data.items()}
groups

{'Abeta Clearance': array(['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11',
        'TI12', 'TI13', 'TI14', 'Abet', 'VC_1', 'VC_2', 'VC_3', 'TI8',
        'TI9', 'TI10', 'TI15'], dtype=object),
 'Abeta secretion': array([2, 3, 4, 'VC', 5, 6, 7, 11, 12, 13, 14, 1, 'RI', 10, 15],
       dtype=object),
 'LPS inflammation': array(['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11',
        'TI12', 'TI13', 'LPS ', 'TI14', 'LPS+', 'VC_1', 'VC_2', 'VC_3',
        'VC_4', 'VC_5', 'Treatment', 'TI 8', 'TI8 ', 'TI 9', 'TI9 ',
        'TI10', 'TI15', 'LPS 1', 'LPS 2', 'LPS 3', 'LPS 4', 'RI 1', 'RI 2',
        'RI 3', 'RI 4', 'VC 1', 'VC 2', 'VC 3', 'VC 4'], dtype=object),
 'Tau phosphorylation': array(['TI1 ', 2, 'TI3', 'TI4', 'TI5', 'TI6', 'TI7', 8, 9, 10, 'TI11',
        'TI12', 'TI13', 'TI14', 15, 'VC', 'RI'], dtype=object)}

In [8]:
treatments = dict()
key = 'Abeta Clearance'
treatments[key] = [x for x in groups[key] if re.match('^TI.*', x)]
print(treatments[key], len(treatments[key]))

['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11', 'TI12', 'TI13', 'TI14', 'TI8', 'TI9', 'TI10', 'TI15'] 15


In [9]:
key = 'Abeta secretion'
treatments[key] = [x for x in groups[key] if isinstance(x, int)]
print(treatments[key], len(treatments[key]))

[2, 3, 4, 5, 6, 7, 11, 12, 13, 14, 1, 10, 15] 13


In [10]:
treatments = dict()
key = 'LPS inflammation'
treatments[key] = [x for x in groups[key] if re.match('^TI[0-9]+', x)]
print(treatments[key], len(treatments[key]))

['TI2 ', 'TI3 ', 'TI4 ', 'TI5 ', 'TI6 ', 'TI7 ', 'TI1 ', 'TI11', 'TI12', 'TI13', 'TI14', 'TI8 ', 'TI9 ', 'TI10', 'TI15'] 15


In [11]:
treatments = dict()
key = 'Tau phosphorylation'
treatments[key] = [x for x in groups[key] if x not in ['VC', 'RI']]
print(treatments[key], len(treatments[key]))

['TI1 ', 2, 'TI3', 'TI4', 'TI5', 'TI6', 'TI7', 8, 9, 10, 'TI11', 'TI12', 'TI13', 'TI14', 15] 15


In [12]:
%connect_info

{
  "shell_port": 53864,
  "iopub_port": 53865,
  "stdin_port": 53866,
  "control_port": 53868,
  "hb_port": 53867,
  "ip": "127.0.0.1",
  "key": "581051de-4f9aad1617a52f40312ba2eb",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-42651906-483d-4596-bc52-5a3c5bb69b29.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
